Train CycleGan

Import Library

In [ ]:
import torch
from torch import nn
from PIL import Image
from torchvision import transforms
import itertools
from torch.utils.data import DataLoader
from torchvision import transforms
import time
import numpy as np
import os
import argparse
import random
from PIL import Image
import torch.utils.data as data
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#def parse_args():
desc = "Pytorch implementation of CinCGAN"
parser = argparse.ArgumentParser(description=desc)
parser.add_argument('--gpu', type=str, default='0', help='GPU idx') #0
parser.add_argument('--inner_lr', type=float, default=2*1e-4, help='initial inner lr')
parser.add_argument('--outer_lr', type=float, default=1e-4, help='initial outer lr')
parser.add_argument('--decay_step', type=int, default=40000, help='decay step')
parser.add_argument('--bsize', type=int, default=16, help='Batch size')
parser.add_argument('--test_bsize', type=int, default=1, help='Test Batch size')
parser.add_argument('--iteration', type=int, default=20, help='iteration') #400000
parser.add_argument('--eps', type=float, default=1e-8, help='Adam eps')
parser.add_argument('--w1', type=float, default=10.0, help='w1')
parser.add_argument('--w2', type=float, default=5.0, help='w2')
parser.add_argument('--w3', type=float, default=0.5, help='w3')
parser.add_argument('--gamma0', type=float, default=1.0, help='gamma0')
parser.add_argument('--gamma1', type=float, default=10.0, help='gamma1')
parser.add_argument('--gamma2', type=float, default=5.0, help='gamma2')
parser.add_argument('--gamma3', type=float, default=2.0, help='gamma3')
parser.add_argument('--gamma4', type=float, default=0.0, help='gamma4')
parser.add_argument('--train_s_path', type=str, default='/content/drive/MyDrive/Data/Train', help='train source dataset path')
parser.add_argument('--train_t_path', type=str, default='/content/drive/MyDrive/Data/Validation', help='train target dataset path')
parser.add_argument('--test_s_path', type=str, default='/content/drive/MyDrive/Test /Test_source', help='test source dataset path')
parser.add_argument('--test_t_path', type=str, default='/content/drive/MyDrive/Test /Test_target', help='test target dataset path')
parser.add_argument('--ngpus_per_node', type=int, default=1, help='ngpus per node')
parser.add_argument('--num_workers', type=int, default=0, help='number of workers') #4
parser.add_argument('--resume', type=bool, default=False, help='resume or not')
parser.add_argument('--resume_iter', type=int, default=1, help='resume iteration')
parser.add_argument('--neptune', type=bool, default=False, help='to use neptune or not')
parser.add_argument('--save_freq', type=int, default=100, help='checkpoint save frequency') #1000
parser.add_argument('--phase', type=str, default='train_inner', help='train_inner / train_outer / test_inner / test_outer')
parser.add_argument('--scale_factor', type=int, default=3, help='scale factor : 2 / 4') #4
parser.add_argument('--inner_ckpt_path', type=str, default='/content/drive/MyDrive/Savings', help='Inner cycle checkpoint file path for fine tuning')
parser.add_argument('--outer_ckpt_path', type=str, default='/content/drive/MyDrive/Savings', help='Outer cycle checkpoint file path for fine tuning')
parser.add_argument('--skip_inner', type=bool, default=False, help='Not update inner cycle')
parser.add_argument('--use_fid', type=bool, default=False, help='Whether to use fid or not')
parser.add_argument('--fid_save_path', type=str, default='/content/drive/MyDrive/Savings', help='Fid save path')
parser.add_argument('--fid_s_path', type=str, default='/content/drive/MyDrive/Data/Train', help='Fid source path')
parser.add_argument('--fid_t_path', type=str, default='/content/drive/MyDrive/Data/Train', help='Fid target path')
parser.add_argument('--fid_bsize', type=int, default=129, help='FID batch size') #128
parser.add_argument('--fid_freq', type=int, default=1000, help='FID frequency')
parser.add_argument('--best_psnr', type=float, default=25.0, help='Best psnr')
parser.add_argument('--ckpt_save_path', type=str, default='/content/drive/MyDrive/checkpoint', help='ckpt save path')
parser.add_argument('--use_psnr', type=bool, default=True, help='Whether to use psnr or not')
parser.add_argument('--psnr_freq', type=int, default=100, help='PSNR frequency')
parser.add_argument('--max_hw', type=int, default=1000, help='Max test input image size')

args = parser.parse_args(args=[])

    #return check_args(parser.parse_args())
'''
def check_args(args):
    try:
        assert args.iteration >= 1
    except:
        print('number of iteration must be larger than or equal to one')

    try:
        assert args.ngpus_per_node == 1
    except:
        print('Multi-gpu is not supported yet')
    return args

 '''

def main_worker_inner(gpu, ngpus_per_node, args):


    gan = CinCGAN(gpu, ngpus_per_node, args)
    gan.test_batch_size = args.test_bsize
    gan.build_model()
    gan.train(inner=True)



Utility and Initialization

In [ ]:
def tensor2pil(t):
    img = transforms.functional.to_pil_image(denorm(t))
    return img
def denorm(t):
    return t*0.5 + 0.5

def tensor_imsave(t, path, fname, denormalization=True, prt=True):
    # Save tensor as .png file
    check_folder(path)
    if denormalization: t = denorm(t)
    t = torch.clamp(input=t, min=0, max=1)
    img = transforms.functional.to_pil_image(t.detach().cpu())
    img.save(os.path.join(path,fname))
    if prt:
        print(f"Saved to {os.path.join(path,fname)}")
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:##Automatic closing using with. rb = read binary
        img = Image.open(f)
        return img.convert('RGB')
def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

class TVLoss(nn.Module):
    def __init__(self,TVLoss_weight=1):
        super(TVLoss,self).__init__()
        self.TVLoss_weight = TVLoss_weight

    def forward(self,x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self._tensor_size(x[:,:,1:,:])
        count_w = self._tensor_size(x[:,:,:,1:])
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        return self.TVLoss_weight*2*(h_tv/count_h+w_tv/count_w)/batch_size

    def _tensor_size(self,t):
        return t.size()[1]*t.size()[2]*t.size()[3]

def pil_batch_loader(root_path, bsize):
    # Returns the list of PIL.Images at given path. Length of list is bsize.
    flist = sorted(glob.glob(root_path, '*.png'))
    flist = flist[:bsize]
    result = []
    for p in flist:
        img = pil_loader(p)
        result.append(img)
    return result

class PSNR(object):
    def __init__(self, gpu, val_max=1, val_min=0, ycbcr=True):
        super(PSNR,self).__init__()
        self.val_max = val_max
        self.val_min = val_min
        self.gpu = gpu
        self.ycbcr = ycbcr

    def __call__(self,x,y):
        """
        if x.is_cuda:
            x = x.detach().cpu()
        if y.is_cuda:
            y = y.detach().cpu()
        """
        assert len(x.size()) == len(y.size())
        with torch.no_grad():
            x_lum = rgb_to_ycbcr(x)[:,0]
            y_lum = rgb_to_ycbcr(y)[:,0]

            mse = torch.mean((y_lum-x_lum)**2, dim=[0,1]) #dim=[1,2]
            psnr = 20*torch.log10(torch.tensor(self.val_max-self.val_min, dtype=torch.float).cuda(self.gpu)) - 10*torch.log10(mse)
            return torch.mean(psnr)

def rgb_to_ycbcr(image):
    r"""Convert an RGB image to YCbCr.

    Args:
        image (torch.Tensor): RGB Image to be converted to YCbCr.

    Returns:
        torch.Tensor: YCbCr version of the image.
    """

    if not torch.is_tensor(image):
        raise TypeError("Input type is not a torch.Tensor. Got {}".format(
            type(image)))

    if len(image.shape) < 3 or image.shape[-3] != 3:
        raise ValueError("Input size must have a shape of (*, 3, H, W). Got {}"
                         .format(image.shape))

    r: torch.Tensor = image[..., 0, :, :]
    g: torch.Tensor = image[..., 1, :, :]
    b: torch.Tensor = image[..., 2, :, :]

    delta = .5
    y: torch.Tensor = .299 * r + .587 * g + .114 * b
    cb: torch.Tensor = (b - y) * .564 + delta
    cr: torch.Tensor = (r - y) * .713 + delta
    return torch.stack((y, cb, cr), -3)


Creating Data Folder

In [ ]:
def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.

    Args:
        filename (string): path to a file

    Returns:
        bool: True if the filename ends with a known image extension
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)

def make_dataset(dir, extensions, label):
    # Returns the tuples (path, label) of data
    # Now, label is same with fname
    images = []
    for root, _, fnames in sorted(os.walk(dir)):
        for fname in sorted(fnames):
            if has_file_allowed_extension(fname, extensions):
                path = os.path.join(root, fname)
                item = (path, fname)
                images.append(item)

    return images

class DatasetFolder(data.Dataset):
    def __init__(self, root, label, extensions = ['.jpg', '.png', '.jpeg', '.bmp'], transform=None, return_two_img = False, big_imsize = 129, scale_factor = 3):
        # root -- Dataset folder path
        # label -- 0 / 1
        # extensions -- list of alowed extensions
        # transform -- Transform applied to image
        # return_two_img -- Whether to return both big and small imgs or not
        # big_imsize -- Size of bigger img. Matters only if return_two_img is True
        # scale_factor -- SR scale factor. Matters only if return_two_img is True
        samples = make_dataset(root, extensions, label)

        if len(samples) == 0:
            raise(RuntimeError("Found 0 files in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(extensions)))

        self.root = root
        self.extensions = extensions
        self.samples = samples
        self.transform = transform
        self.return_two_img = return_two_img
        self.big_imsize = big_imsize
        self.scale_factor = scale_factor
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """

        path, target = self.samples[index]
        sample = pil_loader(path)



        if self.return_two_img:
            t1 = transforms.Compose([
            Random90Rot(),
            transforms.RandomCrop((self.big_imsize,self.big_imsize))])

            t2 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

            sample = t1(sample)
            sample_big = t2(sample)
            sample_small = transforms.Resize((self.big_imsize//self.scale_factor, self.big_imsize//self.scale_factor), Image.BICUBIC)(sample)
            sample_small = t2(sample_small)

            return sample_big, sample_small, target

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

    def __len__(self):
        return len(self.samples)

Generator and Discriminator Networks

In [ ]:
class ResnetGenerator(nn.Module):
    #Generator architecture
    def __init__(self, input_nc=3, output_nc=3, inter_nc=64, n_blocks=6, img_size = 32, use_bias = False, rs_norm = 'BN', padding_type = 'zero', dsple = False, scale_factor=4):
        # input_nc(int) -- The number of channels of input img
        # output_nc(int) -- The number of channels of output img
        # inter_nc(int) -- The number of filters of intermediate layers
        # n_blocks(int) -- The number of resnet blocks
        # img_size(int) -- Input image size
        # use_bias(bool) -- Whether to use bias on conv layer or not
        # rs_norm(str) -- The type of normalization method of ResnetBlock. BN : Batch Normalization, IN : Instance Normalization, else : none
        # padding_type(str) -- The name of padding layer: reflect | replicate | zero
        # dsple(bool) -- Whether to downsample or maintain input image. Set it true for G3.
        # scale_factor(int) -- Scale factor, 2 / 4
        super(ResnetGenerator, self).__init__()

        self.input_nc = input_nc
        self.output_nc = output_nc
        self.inter_nc = inter_nc
        self.n_blocks = n_blocks
        self.img_size = img_size
        self.use_bias = use_bias
        self.rs_norm = rs_norm
        self.padding_type = padding_type
        self.dsple = dsple
        self.scale_factor = scale_factor

        # Input blocks
        InBlock = []

        InBlock += [nn.Conv2d(input_nc, inter_nc, kernel_size=7, stride=1, padding=3, bias=self.use_bias),
                     nn.LeakyReLU(0.2)]
        InBlock += [nn.Conv2d(inter_nc, inter_nc, kernel_size=3, stride=2 if self.dsple and self.scale_factor==4 else 1, padding=1, bias=self.use_bias),
                     nn.LeakyReLU(0.2)] #changed
        InBlock += [nn.Conv2d(inter_nc, inter_nc, kernel_size=3, stride=2 if self.dsple else 1, padding=1, bias=self.use_bias),
                     nn.LeakyReLU(0.2)]

        # ResnetBlocks
        ResnetBlocks = []

        for i in range(n_blocks):
            ResnetBlocks += [ResnetBlock(inter_nc, self.padding_type, self.rs_norm, self.use_bias)]

        # Output block
        OutBlock = []

        OutBlock += [nn.Conv2d(inter_nc, inter_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias),
                     nn.LeakyReLU(0.2)]
        OutBlock += [nn.Conv2d(inter_nc, inter_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias),
                     nn.LeakyReLU(0.2)]
        OutBlock += [nn.Conv2d(inter_nc, output_nc, kernel_size=7, stride=1, padding=3, bias=self.use_bias),
                     nn.LeakyReLU(0.2)]

        self.InBlock = nn.Sequential(*InBlock)
        self.ResnetBlocks = nn.Sequential(*ResnetBlocks)
        self.OutBlock = nn.Sequential(*OutBlock)
    def forward(self,x):
        out = self.InBlock(x)
        out = self.ResnetBlocks(out)
        out = self.OutBlock(out)

        return out
class ResnetBlock(nn.Module):
    def __init__(self, dim, padding_type, norm_type, use_bias):
        # dim(int) -- The number of channels in the resnet blocks
        # padding_type(str) -- The name of padding layer: reflect | replicate | zero
        # norm_type(str) -- The type of normalization method. BN : Batch Normalization, IN : Instance Normalization, else : none
        # use_bias -- Whether to use bias on conv layer or not
        super(ResnetBlock, self).__init__()

        conv_block = []

        # Padding
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        if norm_type=='BN':
            norm_layer = nn.BatchNorm2d
        elif norm_type=='IN':
            norm_layer = nn.InstanceNorm2d
        else:
            raise NotImplementedError('Normalization [%s] is not implemented' % norm_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias),
                       norm_layer(dim),
                       nn.LeakyReLU(0.2)]


        # Padding
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias),
                       norm_layer(dim),
                       nn.LeakyReLU(0.2)]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        out = self.conv_block(x)

        # Skip connection
        out = out + x

        return out


class Discriminator(nn.Module):
    def __init__(self, input_nc=3, norm_type = 'BN', use_bias = True, is_inner=True, scale_factor=4):
        # input_nc(int) -- The number of channels of input img
        # norm_type(str) -- The type of normalization method. BN : Batch Normalization, IN : Instance Normalization, else : none
        # use_bias(bool) -- Whether to use bias or not
        # is_inner(bool) -- True : For inner cycle, False : For outer cycle
        # scale_factor(int) -- Scale factor, 2 / 4

        super(Discriminator, self).__init__()

        if norm_type=='BN':
            norm_layer = nn.BatchNorm2d
            use_bias = False # There is no need to use bias because BN already has shift parameter.
        elif norm_type=='IN':
            norm_layer = nn.InstanceNorm2d
        else:
            raise NotImplementedError('Normalization [%s] is not implemented' % norm_type)

        if is_inner == True:
            s = 1
        elif is_inner == False:
            s = 2
        else:
            raise NotImplementedError('is_inner must be boolean.')

        nfil_mul = 64
        p=0 # Why 1???
        layers = []
        layers += [nn.Conv2d(input_nc, nfil_mul, kernel_size=4, stride = 2 if is_inner==True and scale_factor==2 else s, padding=p, bias=use_bias),
                       nn.LeakyReLU(0.2)] # changed
        layers += [nn.Conv2d(nfil_mul, nfil_mul*2, kernel_size=4, stride = s, padding=p, bias=use_bias),
                       norm_layer(nfil_mul*2),
                       nn.LeakyReLU(0.2)]
        layers += [nn.Conv2d(nfil_mul*2, nfil_mul*4, kernel_size=4, stride = s, padding=p, bias=use_bias),
                       norm_layer(nfil_mul*4),
                       nn.LeakyReLU(0.2)]
        layers += [nn.Conv2d(nfil_mul*4, nfil_mul*8, kernel_size=4, stride = 1, padding=p, bias=use_bias),
                       norm_layer(nfil_mul*8),
                       nn.LeakyReLU(0.2)]
        layers += [nn.Conv2d(nfil_mul*8, 1, kernel_size=4, stride = 1, padding=p, bias=use_bias),
                       nn.LeakyReLU(0.2)]
        self.layers = nn.Sequential(*layers)

    def forward(self,x):
        out = self.layers(x)

        return out # Predicted values of each patches

Inner Cycle Model

In [ ]:

class Crop(object):
    def __init__(self, max_hw):
        self.max_hw = max_hw
    def __call__(self,x):
        if x.size[0]>=self.max_hw and x.size[1]>=self.max_hw:
            x = transforms.CenterCrop((self.max_hw,self.max_hw))(x)
        return x

class Random90Rot(object):
    def __call__(self, img):
        angle_list = [0,-90,-180,90]
        angle = random.choice(angle_list)

        return transforms.functional.rotate(img, angle)

In [ ]:
'''
Computing mean and std of dataset
mean = 0.0
meansq = 0.0
count = 0

for index, data in enumerate(train_loader):
    mean = data.sum()
    meansq = meansq + (data**2).sum()
    count += np.prod(data.shape)

total_mean = mean/count
total_var = (meansq/count) - (total_mean**2)
total_std = torch.sqrt(total_var)
print("mean: " + str(total_mean))
print("std: " + str(total_std))
'''

'\nComputing mean and std of dataset\nmean = 0.0\nmeansq = 0.0\ncount = 0\n\nfor index, data in enumerate(train_loader):\n    mean = data.sum()\n    meansq = meansq + (data**2).sum()\n    count += np.prod(data.shape)\n\ntotal_mean = mean/count\ntotal_var = (meansq/count) - (total_mean**2)\ntotal_std = torch.sqrt(total_var)\nprint("mean: " + str(total_mean))\nprint("std: " + str(total_std))\n'

In [ ]:
class CinCGAN():
    def __init__(self, gpu, ngpus_per_node, args):
        # Device configuration

        torch.cuda.set_device(gpu)

        # self.device = torch.device('cuda:'+ args.gpu if torch.cuda.is_available() else 'cpu')
        self.device = torch.device('cpu')

        print("GPU : ", gpu)
        self.lr = args.inner_lr
        self.decay_step = args.decay_step
        self.batch_size = args.bsize//ngpus_per_node
        self.test_batch_size = args.test_bsize
        print(f"GPU : {gpu}, BatchSize : {self.batch_size}")
        self.iteration = args.iteration
        self.eps = args.eps
        self.w1 = args.w1
        self.w2 = args.w2
        self.w3 = args.w3

        self.lr_outer = args.outer_lr

        self.gamma0 = args.gamma0
        self.gamma1 = args.gamma1
        self.gamma2 = args.gamma2
        self.gamma3 = args.gamma3
        self.gamma4 = args.gamma4

        self.train_s_path = args.train_s_path
        self.train_t_path = args.train_t_path
        self.test_s_path = args.test_s_path
        self.test_t_path = args.test_t_path
        self.fid_s_path = self.test_s_path
        self.gpu = gpu
        self.ngpus_per_node = ngpus_per_node
        self.num_workers = args.num_workers

        self.step = 0

        self.resume = args.resume
        self.resume_iter = args.resume_iter
        self.inner_ckpt_path = args.inner_ckpt_path
        self.neptune = args.neptune
        self.params = vars(args)



        self.save_freq = args.save_freq

        self.scale_factor = args.scale_factor
        self.skip_inner = args.skip_inner
        self.use_fid = args.use_fid
        self.fid_save_path = args.fid_save_path
        #self.fid_s_path = args.fid_s_path
        #self.fid_t_path = args.fid_t_path
        #self.fid_bsize = args.fid_bsize
        #self.fid_freq = args.fid_freq
        #self.fid_mean = None
        #self.fid_cov = None
        self.use_psnr = args.use_psnr
        self.psnr_freq = args.psnr_freq
        self.ckpt_save_path = args.ckpt_save_path
        if os.path.isdir('/mnt/workspace/CinCGAN-pytorch/CinCGAN_pytorch/target_fid_statistics'):
            self.fid_mean = np.load(os.path.join('/mnt/workspace/CinCGAN-pytorch/CinCGAN_pytorch/target_fid_statistics', 'm2.npy'))
            self.fid_cov = np.load(os.path.join('/mnt/workspace/CinCGAN-pytorch/CinCGAN_pytorch/target_fid_statistics', 's2.npy'))


        self.inception = None
        self.best_psnr = args.best_psnr

        self.imsize_y = 129 ##128
        assert self.imsize_y % self.scale_factor == 0, 'invalid scale factor'
        self.imsize_x = 128//self.scale_factor
        self.max_hw = args.max_hw
        print(f"NEPTUNE : {self.neptune}")
        print(f"gamma0 : {self.gamma0}")
        print(f"gamma1 : {self.gamma1}")
        print(f"gamma2 : {self.gamma2}")
        print(f"gamma3 : {self.gamma3}")
        print(f"gamma4 : {self.gamma4}")
        print(f"train_s_path : {self.train_s_path}")
        print(f"train_t_path : {self.train_t_path}")

        print(f"test_s_path : {self.test_s_path}")
        print(f"test_t_path : {self.test_t_path}")
        print(f"scale factor : {self.scale_factor}")

    def psnr_mean(self, inner, num=10):
        # Derive mean of psnr of whole test data
        # num : int, number of samples
        test_s_iter = iter(self.test_s_loader)
        test_t_iter = iter(self.test_t_loader)
        num = np.minimum(num, len(self.test_s_loader))
        self.G1_forward.eval()
        psnr_mean = 0
        i=0
        with torch.no_grad():
            while True:
                try:
                    x, _ = test_s_iter.next()
                except:
                    break
                try:
                    y_big, _ = test_t_iter.next()
                except:
                    break
                y = transforms.Resize(x.size()[-2:], interpolation=Image.BICUBIC)(tensor2pil(y_big[0])) # To resize, tensor should be converted to pillow image.
                y = transforms.ToTensor()(y)
                y = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(y)
                y = torch.unsqueeze(y,0)
                y = y.to(self.device)

                x, y_big = x.to(self.device), y_big.to(self.device)
                # fake_inner = self.G1_forward(x)
                out = self.G1_forward(x)
                if inner == True:
                    out = out * 0.5 + 0.5
                    psnr = self.PSNR(y*0.5 + 0.5, out)
                psnr_mean += psnr/num
                print(f"i : {i}, psnr : {psnr:0.4f}")
                i+=1
                if i==num:
                    break
        return psnr_mean

    def save(self, is_outer=True, fname = 'default'):
        if fname == 'default':
            fname = f'{self.step}.pt'
        check_folder(os.path.join(self.ckpt_save_path, 'ckpt'))
        params = {}
        params['G1_forward'] = self.G1_forward.state_dict()
        params['G1_backward'] = self.G1_backward.state_dict()
        params['D1_forward'] = self.D1_forward.state_dict()
        params['D1_backward'] = self.D1_backward.state_dict()

        torch.save(params, os.path.join(self.ckpt_save_path, 'ckpt', fname))
        if is_outer==True:
            check_folder(os.path.join(self.ckpt_save_path, 'ckpt_edsr'))
            params_edsr = {}
            params_edsr['EDSR'] = self.EDSR.state_dict()
            torch.save(params_edsr, os.path.join(self.ckpt_save_path,'ckpt_edsr', fname))

    def build_model(self):


        """ Define Inner Cycle Models """


        self.G1_forward = ResnetGenerator(scale_factor = self.scale_factor).to(self.device)
        self.G1_backward = ResnetGenerator(scale_factor = self.scale_factor).to(self.device)
        self.D1_forward = Discriminator(scale_factor = self.scale_factor).to(self.device)
        self.D1_backward = Discriminator(scale_factor = self.scale_factor).to(self.device)

        """ Define Loss """
        self.MSE_loss = nn.MSELoss().to(self.device)
        self.TV_loss= TVLoss().to(self.device)

        self.PSNR = PSNR(self.gpu)

        """ Optimizers """
        self.G_optim = torch.optim.Adam(itertools.chain(self.G1_forward.parameters(), self.G1_backward.parameters()),lr=self.lr, betas=(0.5,0.999), eps = self.eps)
        self.D_optim = torch.optim.Adam(itertools.chain(self.D1_forward.parameters(), self.D1_backward.parameters()),lr=self.lr, betas=(0.5,0.999), eps = self.eps)

        """ Data loader """

        train_transform_s = transforms.Compose([
            Random90Rot(),
            transforms.RandomCrop((self.imsize_x,self.imsize_x)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

        train_transform_t = transforms.Compose([
            Random90Rot(),
            transforms.RandomCrop((self.imsize_y,self.imsize_y)),
            transforms.Resize((self.imsize_x,self.imsize_x), Image.BICUBIC), ###
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])



        test_transform = transforms.Compose([
            Crop(self.max_hw),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])


        self.train_s_dataset = DatasetFolder(self.train_s_path, 0, transform = train_transform_s)
        self.train_t_dataset = DatasetFolder(self.train_t_path, 1, transform=None, return_two_img = True, big_imsize=self.imsize_y, scale_factor = self.scale_factor)

        self.test_s_dataset = DatasetFolder(self.test_s_path, 0, transform=test_transform)
        self.test_t_dataset = DatasetFolder(self.test_t_path, 1, transform=test_transform)

        self.train_s_loader = DataLoader(self.train_s_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        self.train_t_loader = DataLoader(self.train_t_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

        self.test_s_loader = DataLoader(self.test_s_dataset, batch_size=self.test_batch_size, shuffle=False, num_workers=self.num_workers)
        self.test_t_loader = DataLoader(self.test_t_dataset, batch_size=self.test_batch_size, shuffle=False, num_workers=self.num_workers)

    def train(self, inner=True):
        psnr = 0
        psnr_inner = 0
        check_g3 = 0
        fid_inner = 0
        fid_outer = 0
        self.inner = inner
        # torch.autograd.set_detect_anomaly(True)
        for model in [self.G1_forward, self.G1_backward, self.D1_forward, self.D1_backward]:
            model.train()
        if self.skip_inner:
            self.G1_forward.eval()
        start_iter = 0
        print(f"NEPTUNE : {self.neptune}")
        if self.neptune:
            neptune.init(project_qualified_name='ml.swlee/CinCGAN',
                        api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYmVhNmFlOGMtZDRmZS00NzIyLWJkYzgtNTcyZTk0ZTM5YzM1In0=')
            neptune.create_experiment(params = self.params)
        if self.resume:
            start_iter = self.resume_iter
            if inner:
                self.load(path_inner = self.inner_ckpt_path)

        # Validation data
        dataiter = iter(self.test_s_loader)
        x_test, _  = next(dataiter)
        #x_test, _ = iter(self.test_s_loader).next()
        #x_test = x_test.to(self.device)

        dataiter1 = iter(self.test_t_loader)
        y_test_big, _  = next(dataiter1)
        #y_test_big, _ = iter(self.test_t_loader).next()
        # if self.use_fid == False:
        #     assert x_test.size()[0] == y_test_big.size()[0] == 1
        y_test = transforms.Resize(x_test.size()[-2:], interpolation=Image.BICUBIC)(tensor2pil(y_test_big[0])) # To resize, tensor should be converted to pillow image.
        y_test = transforms.ToTensor()(y_test)
        y_test = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(y_test)
        y_test = torch.unsqueeze(y_test,0)
        y_test = y_test.to(self.device)
        y_test_big = y_test_big.to(self.device)



        if self.use_psnr: base_d = self.PSNR(denorm(x_test[0]), denorm(y_test[0])) ###
        if self.ngpus_per_node>1:
            if self.gpu==0:
                x_test, _ = iter(self.test_s_loader).next()
                x_test = x_test.to(self.device)

                y_test_big, _ = iter(self.test_t_loader).next()
                assert x_test.size()[0] == y_test_big.size()[0] == 1
                y_test = transforms.Resize(x_test.size()[-2:], interpolation=Image.BICUBIC)(tensor2pil(y_test_big[0])) # To resize, tensor should be converted to pillow image.
                y_test = torch.unsqueeze(transforms.ToTensor()(y_test),0)
                y_test = y_test.to(self.device)


                if self.use_psnr: base_d = self.PSNR(x_test*0.5 + 0.5, y_test*0.5 + 0.5)



        if self.skip_inner:
            freeze_model(self.G1_forward)
            print("----------------------------Model Freezed")


        for step in range(start_iter, self.iteration+1):
            self.step = step
            t = time.time()
            if step%self.decay_step==0:
                n = step//self.decay_step
                self.G_optim.param_groups[0]['lr'] = self.lr/2**n # param_groups = [{'amsgrad': False, 'betas': (...), 'eps': 1e-08, 'lr': 0.0002, 'params': [...], 'weight_decay': 0}]
                self.D_optim.param_groups[0]['lr'] = self.lr/2**n
                #self.G_outer_optim.param_groups[0]['lr'] = self.lr_outer/2**n
                #self.D2_optim.param_groups[0]['lr'] = self.lr_outer/2**n


            dataiter2 = iter(self.train_s_loader)
            x, label_x  = next(dataiter2)

            dataiter3 = iter(self.train_t_loader)
            y_big, y, label_y  = next(dataiter3)
            '''
            try:
                x, label_x = train_s_iter.next()
            except:
                train_s_iter = iter(self.train_s_loader)
                x, label_x = train_s_iter.next()

            try:
                y_big, y, label_y = train_t_iter.next()

            except:
                train_t_iter = iter(self.train_t_loader)
                y_big, y, label_y = train_t_iter.next()

            '''
            if self.ngpus_per_node>1:
                x, y = x.to(self.device), y.to(self.device)



            x, y = x.to(self.device), y.to(self.device)


            y_big = y_big.to(self.device)
            print(f"x.shape : {x.size()}, y.shape : {y.size()}, y_big.shape : {y_big.size()}")




            if self.gpu==0: print(f"Data loading : {time.time()-t:.3f}")
            t = time.time()
            # Update discriminator
            self.D_optim.zero_grad()
            if self.skip_inner==False:


                fake_forward = self.G1_forward(x)
                fake_backward = self.G1_backward(y)
                print(f"fake_forward.shape : {fake_forward.size()} fake_backward.shape : {fake_backward.size()} ")
                # D1 output for real images
                dis1_forward_output_real = self.D1_forward(y)
                dis1_backward_output_real = self.D1_backward(x)

                # D1 output for fake images
                dis1_forward_output_fake = self.D1_forward(fake_forward)
                dis1_backward_output_fake = self.D1_backward(fake_backward)
                print(f"dis1_forward_output_fake.shape : {dis1_forward_output_fake.size()} dis1_backward_output_fake.shape : {dis1_backward_output_fake.size()} ")
                # D1 prediction for real images
                dis1_forward_predicted_real = torch.mean(dis1_forward_output_real, dim = [1,2,3])
                dis1_backward_predicted_real = torch.mean(dis1_backward_output_real, dim = [1,2,3])

                # D1 prediction for fake images
                dis1_forward_predicted_fake = torch.mean(dis1_forward_output_fake, dim = [1,2,3])
                dis1_backward_predicted_fake = torch.mean(dis1_backward_output_fake, dim = [1,2,3])



                # Discriminator loss
                dis1_forward_ad_loss = self.MSE_loss(dis1_forward_predicted_fake, torch.zeros_like(dis1_forward_predicted_fake).to(self.device)) + self.MSE_loss(dis1_forward_predicted_real, torch.ones_like(dis1_forward_predicted_real).to(self.device))
                dis1_backward_ad_loss = self.MSE_loss(dis1_backward_predicted_fake, torch.zeros_like(dis1_backward_predicted_fake).to(self.device)) + self.MSE_loss(dis1_backward_predicted_real, torch.ones_like(dis1_backward_predicted_real).to(self.device))

                dis1_loss = dis1_forward_ad_loss + dis1_backward_ad_loss

                dis1_loss.backward()
                self.D_optim.step()



                # Update Generator
                self.G_optim.zero_grad()

                fake_forward = self.G1_forward(x)
                fake_backward = self.G1_backward(y)

                # D1 output for fake images
                dis1_forward_output_fake = self.D1_forward(fake_forward)
                dis1_backward_output_fake = self.D1_backward(fake_backward)

                # D1 prediction for fake images
                dis1_forward_predicted_fake = torch.mean(dis1_forward_output_fake, dim = [1,2,3])
                dis1_backward_predicted_fake = torch.mean(dis1_backward_output_fake, dim = [1,2,3])

                # Adversarial loss
                g1_forward_ad_loss = self.MSE_loss(dis1_forward_predicted_fake, torch.ones_like(dis1_forward_predicted_fake).to(self.device))
                g1_backward_ad_loss = self.MSE_loss(dis1_backward_predicted_fake, torch.ones_like(dis1_backward_predicted_fake).to(self.device))

                # Cycle consistency loss
                g1_forward_cycle_loss = self.MSE_loss(self.G1_backward(fake_forward), x)
                g1_backward_cycle_loss = self.MSE_loss(self.G1_forward(fake_backward), y)

                # Identity loss
                g1_forward_identity_loss = self.MSE_loss(self.G1_forward(y), y)
                g1_backward_identity_loss = self.MSE_loss(self.G1_backward(x), x)

                # Total variation loss
                g1_forward_tv_loss = self.TV_loss(fake_forward)

                if inner==False:
                    self.w2 = 1

                g1_forward_loss = g1_forward_ad_loss + self.w1*g1_forward_cycle_loss + self.w2*g1_forward_identity_loss + self.w3*g1_forward_tv_loss
                g1_backward_loss = g1_backward_ad_loss + self.w1*g1_backward_cycle_loss + self.w2*g1_backward_identity_loss

                g1_loss = g1_forward_loss + g1_backward_loss

                g1_loss.backward()
                self.G_optim.step()

                if inner==True:
                    print(f"endloop : {time.time()-t:.3f}")
                    t = time.time()

                    print(f"step : {step}  d_loss : {dis1_loss}  g_loss : {g1_loss}  lr : {self.D_optim.param_groups[0]['lr']}")
            # get inner psnr

            self.G1_forward.eval()

            with torch.no_grad():
                img_t = self.G1_forward(x_test)[0]
                if self.use_psnr and step%self.psnr_freq==0:
                    #psnr = self.PSNR(img_t*0.5 + 0.5, y_test[0]*0.5 + 0.5)
                    psnr_inner = self.psnr_mean(inner=True)
                    print(f"inner psnr : {psnr_inner}")
                    if inner:
                        if psnr_inner > self.best_psnr:
                            self.best_psnr = psnr_inner
                            self.save(is_outer = False, fname = f'inner_{step}_{psnr_inner.item():0.4f}.pt')
            '''
            if inner==True and self.use_fid==True:
                if step%self.fid_freq==0:
                    fid_inner = self.fid()
'''
            if self.skip_inner == False:
                self.G1_forward.train()
            if self.neptune:
                if self.use_psnr: neptune.log_metric('inner_psnr', psnr_inner)
                if self.use_fid: neptune.log_metric('inner_fid', fid_inner)

            if inner==True:
                if step%50==0:

                    tensor_imsave(img_t, f'/mnt/workspace/CinCGAN-pytorch/CinCGAN_pytorch/imgs', f'img{step}.png')
                if self.skip_inner == False: self.G1_forward.train()

            if inner==True:
                if step%self.save_freq == 0:
                    self.save(is_outer=False)
                continue


In [ ]:
def main():
    #args = parse_args()

    gpu = args.gpu
    ngpus_per_node = args.ngpus_per_node

    #os.environ["CUDA_VISIBLE_DEVICES"]=gpu
    phase =args.phase
    if phase == 'train_inner' or phase == 'train_outer':

        main_worker = main_worker_inner if phase=='train_inner' else main_worker_outer
        # ngpus_per_node = torch.cuda.device_count()
        world_size = ngpus_per_node
        print("Available gpu num : ", ngpus_per_node)

        if ngpus_per_node >1:
            torch.multiprocessing.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, ))
        else:
            main_worker(int(gpu), 1, args)


    elif phase == 'test_inner' or phase == 'test_outer':

        print('----------Test mode----------')
        gan = CinCGAN(int(gpu), 1, args)
        gan.build_model()
        gan.load(args.inner_ckpt_path, inner= phase=='test_inner', path_outer=args.outer_ckpt_path)
        for i in range(18):
            gan.test(inner= phase=='test_inner', idx=i)
        i = 0
        # print(gan.psnr_mean(inner=False, num=800))
        # result = gan.psnr_mean()
        # print(f"Mean PSNR : {result}")
if __name__ == '__main__':
    main()

Available gpu num :  1
GPU :  0
GPU : 0, BatchSize : 16
NEPTUNE : False
gamma0 : 1.0
gamma1 : 10.0
gamma2 : 5.0
gamma3 : 2.0
gamma4 : 0.0
train_s_path : /content/drive/MyDrive/Data/Train
train_t_path : /content/drive/MyDrive/Data/Validation
test_s_path : /content/drive/MyDrive/Test /Test_source
test_t_path : /content/drive/MyDrive/Test /Test_target
scale factor : 3
NEPTUNE : False
x.shape : torch.Size([16, 3, 42, 42]), y.shape : torch.Size([16, 3, 43, 43]), y_big.shape : torch.Size([16, 3, 129, 129])
Data loading : 0.272
fake_forward.shape : torch.Size([16, 3, 42, 42]) fake_backward.shape : torch.Size([16, 3, 43, 43]) 
dis1_forward_output_fake.shape : torch.Size([16, 1, 27, 27]) dis1_backward_output_fake.shape : torch.Size([16, 1, 28, 28]) 
endloop : 25.995
step : 0  d_loss : 1.4340181350708008  g_loss : 10.830108642578125  lr : 0.0002
inner psnr : 0
Saved to /mnt/workspace/CinCGAN-pytorch/CinCGAN_pytorch/imgs/img0.png
x.shape : torch.Size([16, 3, 42, 42]), y.shape : torch.Size([16, 3,

In [ ]:
test_transform = transforms.Compose([
            Crop(1000),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])
test_s_dataset = DatasetFolder(args.test_s_path, 0, transform=test_transform)
test_s_loader = DataLoader(test_s_dataset, batch_size=1, shuffle=False, num_workers=0)
test_t_dataset = DatasetFolder(args.test_t_path, 1, transform=test_transform)
test_t_loader = DataLoader(test_t_dataset, batch_size=1, shuffle=False, num_workers=0)

dataiter = iter(test_s_loader)
x_test, _  = next(dataiter)
dataiter1 = iter(test_t_loader)
y_test_big, _  = next(dataiter1)

y_test = transforms.Resize(x_test.size()[-2:], interpolation=Image.BICUBIC)(tensor2pil(y_test_big[0])) # To resize, tensor should be converted to pillow image.
y_test = transforms.ToTensor()(y_test)
y_test = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(y_test)
y_test = torch.unsqueeze(y_test,0)
print(y_test[0].size())
print(x_test[0].size())


torch.Size([3, 480, 492])
torch.Size([3, 480, 492])


/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [ ]:
x = torch.randn(3,480,490)
#x_lum = rgb_to_ycbcr(x)[:,0]
(x[:,0]).size()

torch.Size([3, 490])